<a href="https://colab.research.google.com/github/fullstack65/Milestones/blob/trunk/compare_softmax1_lama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -q torch numpy transformers datasets tiktoken wandb tqdm sentencepiece

In [ ]:
# !git clone https://github.com/tcapelle/llama2.c

In [ ]:
import wandb
# wandb.login()

In [ ]:
%cd llama2.c

/content/llama2.c


In [ ]:
import torch

run = wandb.init()

# Softmax1:
# artifact = run.use_artifact('capecape/llamac/jvvylljz_model:v29', type='model')

# Baseline
# artifact = run.use_artifact('capecape/llamac/2q4sljmu_model:v14', type='model')
artifact = run.use_artifact('capecape/llamac/2q4sljmu_model:v0', type='model')


# New run
# artifact = run.use_artifact('capecape/llamac/na2dcgu6_model:v0', type='model')


artifact_dir = artifact.download()


ckpt = torch.load(artifact_dir+'/ckpt.pt')
ckpt.keys()

wandb: Currently logged in as: johnowhitaker. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact 2q4sljmu_model:v0, 1301.59MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:1:38.0


dict_keys(['model', 'optimizer', 'model_args', 'iter_num', 'best_val_loss', 'config'])

In [ ]:
from model import Transformer, ModelArgs

# Settings
model_args = dict(
    softmax=ckpt['config']['softmax'], # softmax or softmax1
    flash=ckpt['config']['flash'],
)
# Additional model args
checkpoint_model_args = ckpt["model_args"]
for k in ["dim", "n_layers", "n_heads", "n_kv_heads",
          "vocab_size", "multiple_of", "max_seq_len"]:
    model_args[k] = checkpoint_model_args[k]

# Load the model
gptconf = ModelArgs(**model_args)
model = Transformer(gptconf)
state_dict = ckpt["model"]

# fix the keys of the state dictionary - needed?
unwanted_prefix = "_orig_mod."
for k, v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix) :]] = state_dict.pop(k)
model.load_state_dict(state_dict);
model.to("cuda");
model.eval();
print(model_args)

{'softmax': 'softmax', 'flash': False, 'dim': 768, 'n_layers': 12, 'n_heads': 12, 'n_kv_heads': 12, 'vocab_size': 32000, 'multiple_of': 32, 'max_seq_len': 1024}


In [ ]:
# Load the dataset
from tinystories import Task
from functools import partial

iter_batches = partial(
    Task.iter_batches,
    batch_size=ckpt['config']['batch_size'],
    max_seq_len=ckpt['config']['max_seq_len'],
    device='cuda',
    num_workers=0,
)

train_batch_iter = iter_batches("data00") # Need to run data prep script or upload the .bin file(s) into /content/llama2.c/data/TinyStories_all_data
X, Y = next(train_batch_iter)

X.shape, Y.shape

Created a PretokDataset with rng seed 42


(torch.Size([32, 1024]), torch.Size([32, 1024]))

In [ ]:
def hook_fn(module, input, output):
    "Hook function to store outputs in a .output attribute"
    module.output = output

hook_handles = []
for b in model.layers:
    hook = b.attention.register_forward_hook(hook_fn)
    hook_handles.append(hook)

In [ ]:
# Collecting some activations
# (NB a batch might be enough to get the stats we want so this is overkill)
acts = []

for i in range(3):
    a = []
    with torch.no_grad():
        X, Y = next(train_batch_iter)
        o, _ = model(X)
    for b in model.layers:
        a.append(b.attention.output.cpu().unsqueeze(0))
    acts.append(torch.cat(a, dim=0))

outputs = torch.cat(acts, dim=1)
outputs.shape



torch.Size([12, 96, 1024, 768])

In [ ]:
# Calculate the kurtosis of all the activations in the model
from scipy.stats import kurtosis
print(kurtosis(outputs.flatten()))

3.8936220105233286


In [ ]:
# Mean per layer
torch.mean(torch.tensor([kurtosis(o.flatten().cpu().float()) for o in outputs]))

tensor(31.3825)

In [ ]:
# Inf norm
max([o.abs().max() for o in outputs])

tensor(11.9557)

In [ ]:
# Baseline at 4k (v0)
# k(all) 3.9
# k(avg) 31.3
# inf norm 12


# S1e-6 at 4k (v0)
# k(all) 4.3
# k(avg) 42.3
# inf norm 8

In [ ]:
# Stats on 3 batches of tinystories (92 examples total, 1024 tokens max in each ):

# Softmax1:
# Inf norm: 24.8
# Avg Kurtosis per layer: 95

# Baseline:
# Inf norm: 31.2
# Avg Kurtosis per layer: 30 (14 across all activarions)

In [ ]:
# # Softmax1:
# [kurtosis(o.flatten().cpu().float()) for o in outputs]
# [108.13075949093135,
#  95.46966161052401,
#  801.2906494227926, # <<<< just one layer that throws things off
#  57.70108403813606,
#  5.221310463381725,
#  3.0735979864031826,
#  1.2426456030982473,
#  1.2678221315969953,
#  2.382823072152604,
#  3.4650911972867773,
#  4.232136406924883,
#  59.24261344775612]

In [ ]:
# # Baseline:
# [kurtosis(o.flatten().cpu().float()) for o in outputs]
# [112.08623327119747,
#  31.946602828938644,
#  169.22754029100614,
#  13.009267154142002,
#  3.9934846124822103,
#  2.065320089852139,
#  1.377368844087628,
#  1.2992505599665511,
#  1.7043503114501695,
#  2.541497231649754,
#  2.893608796813403,
#  11.769736114437878]